# Load modules

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV

import shap 
%matplotlib notebook
%load_ext tensorboard

# Load data

In [3]:
data = np.load(os.path.join('data', 'feature_extracted', 'feature_extracted_dataset_1.npz'), allow_pickle = True)

#training dataset 
x_train = data['x_train']
y_train = data['y_train']
idx_train = data['idx_train']

#test dataset 
x_test = data['x_test']
idx_test = data['idx_test']

# Model building

In [98]:
inp1 = tf.keras.layers.Input(shape=(deep_features_train['ECG_features_C'].shape[1], deep_features_train['ECG_features_C'].shape[2]))
inp2 = tf.keras.layers.Input(shape=(deep_features_train['ECG_features_T'].shape[1], deep_features_train['ECG_features_T'].shape[2]))
inp3 = tf.keras.layers.Input(shape=(handcrafted_features_train['ECG_features'].shape[1], handcrafted_features_train['ECG_features'].shape[2]))
inp4 = tf.keras.layers.Input(shape=(handcrafted_features_train['GSR_features'].shape[1], handcrafted_features_train['GSR_features'].shape[2]))

n_units = 32
n_batch = 64 
recurrent_dropout = 0.5
dropout = 0.2
optimizer = 'rmsprop'

two_layers = True
three_layers = True

#x = tf.keras.layers.Masking(mask_value=0.)(inp1)
#x = tf.keras.layers.GRU(n_units,recurrent_dropout=recurrent_dropout)(x)
#x = tf.keras.layers.Dropout(dropout)(x)
#x = tf.keras.layers.Dense(1)(x)


x = tf.keras.layers.Masking(mask_value=0.)(inp1)
x = tf.keras.layers.GRU(n_units, dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=two_layers or three_layers)(x)
if two_layers or three_layers:
    x = tf.keras.layers.GRU(int(n_units/2), dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=three_layers)(x)
if three_layers:
    x = tf.keras.layers.GRU(int(n_units/4), dropout = dropout, recurrent_dropout=recurrent_dropout)(x)

y = tf.keras.layers.Masking(mask_value=0.)(inp2)
y = tf.keras.layers.GRU(n_units, dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=two_layers or three_layers)(y)
if two_layers or three_layers:
    y = tf.keras.layers.GRU(int(n_units/2), dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=three_layers)(y)
if three_layers:
    y = tf.keras.layers.GRU(int(n_units/4), dropout = dropout, recurrent_dropout=recurrent_dropout)(y)

z = tf.keras.layers.Masking(mask_value=0.)(inp3)
z = tf.keras.layers.GRU(n_units, dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=two_layers or three_layers)(z)
if two_layers or three_layers:
    z = tf.keras.layers.GRU(int(n_units/2), dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=three_layers)(z)
if three_layers:
    z = tf.keras.layers.GRU(int(n_units/4), dropout = dropout, recurrent_dropout=recurrent_dropout)(z)

w = tf.keras.layers.Masking(mask_value=0.)(inp4)
w = tf.keras.layers.GRU(n_units, dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=two_layers or three_layers)(inp4)
if two_layers or three_layers:
    w = tf.keras.layers.GRU(int(n_units/2), dropout = dropout, recurrent_dropout=recurrent_dropout,return_sequences=three_layers)(w)
if three_layers:
    w = tf.keras.layers.GRU(int(n_units/4), dropout = dropout, recurrent_dropout=recurrent_dropout)(w)
#w = tf.keras.layers.Dense(1)(w)

c = tf.keras.layers.Concatenate()([
    #x,
    y,
    z, 
    w
])

c =  tf.keras.layers.Dense(16, activation='sigmoid')(c)

out =  tf.keras.layers.Dense(1, activation='sigmoid')(c)

model = tf.keras.models.Model(inputs=[
        #inp1,
        inp2,
        inp3,
        inp4
        ],
        outputs=out)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

print(model.summary())

model.fit(x=[
            #deep_features_train['ECG_features_C'],
            deep_features_train['ECG_features_T'],
            handcrafted_features_train['ECG_features'],
            handcrafted_features_train['GSR_features']
            ], 
          y=y_train, 
          epochs=100,
          validation_split=0.2,
          callbacks=[tensorboard_callback],
          batch_size = n_batch)


Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_82 (InputLayer)          [(None, 60, 64)]     0           []                               
                                                                                                  
 input_83 (InputLayer)          [(None, 60, 8)]      0           []                               
                                                                                                  
 masking_56 (Masking)           (None, 60, 64)       0           ['input_82[0][0]']               
                                                                                                  
 masking_57 (Masking)           (None, 60, 8)        0           ['input_83[0][0]']               
                                                                                           

Epoch 86/100
26/26 [==============================] - 5s 172ms/step - loss: 0.5721 - accuracy: 0.6890 - val_loss: 0.5741 - val_accuracy: 0.6836
Epoch 87/100
26/26 [==============================] - 4s 166ms/step - loss: 0.5738 - accuracy: 0.6914 - val_loss: 0.5695 - val_accuracy: 0.7150
Epoch 88/100
26/26 [==============================] - 4s 164ms/step - loss: 0.5691 - accuracy: 0.6890 - val_loss: 0.5670 - val_accuracy: 0.7029
Epoch 89/100
26/26 [==============================] - 4s 165ms/step - loss: 0.5614 - accuracy: 0.7065 - val_loss: 0.5671 - val_accuracy: 0.7029
Epoch 90/100
26/26 [==============================] - 4s 167ms/step - loss: 0.5730 - accuracy: 0.6969 - val_loss: 0.5802 - val_accuracy: 0.7174
Epoch 91/100
26/26 [==============================] - 5s 174ms/step - loss: 0.5717 - accuracy: 0.6884 - val_loss: 0.5652 - val_accuracy: 0.7005
Epoch 92/100
26/26 [==============================] - 5s 184ms/step - loss: 0.5662 - accuracy: 0.7011 - val_loss: 0.5836 - val_accuracy:

In [99]:
%tensorboard --logdir logs/fit

In [100]:
model = tf.keras.models.Model(inputs=[
        #inp1,
        inp2,
        inp3,
        inp4
        ],
        outputs=out)

model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

print(model.summary())

model.fit(x=[
            #deep_features_train['ECG_features_C'],
            deep_features_train['ECG_features_T'],
            handcrafted_features_train['ECG_features'],
            handcrafted_features_train['GSR_features']
            ], 
          y=y_train, 
          epochs=50,
          callbacks=[tensorboard_callback],
          batch_size = n_batch)

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_82 (InputLayer)          [(None, 60, 64)]     0           []                               
                                                                                                  
 input_83 (InputLayer)          [(None, 60, 8)]      0           []                               
                                                                                                  
 masking_56 (Masking)           (None, 60, 64)       0           ['input_82[0][0]']               
                                                                                                  
 masking_57 (Masking)           (None, 60, 8)        0           ['input_83[0][0]']               
                                                                                           

# Make and save predictions

In [108]:
y_hat_test = model.predict([
            #deep_features_test['ECG_features_C'],
            deep_features_test['ECG_features_T'],
            handcrafted_features_test['ECG_features'],
            handcrafted_features_test['GSR_features']
            ])
print(y_hat_test)
th = 0.5
#y_hat_test[np.argwhere(np.isnan(y_hat_test))]= 0
y_hat_test[y_hat_test >= th] = 1
y_hat_test[y_hat_test < th] = 0
np.savetxt(os.path.join('submission', 'answer.txt'),y_hat_test.astype(int),fmt='%1d')

31/31 [==============================] - 1s 21ms/step
[[0.81311905]
 [0.8748807 ]
 [0.73365885]
 [0.9043902 ]
 [0.78710985]
 [0.85136294]
 [0.64858943]
 [0.6342362 ]
 [0.71090853]
 [0.68981034]
 [0.69976306]
 [0.54497564]
 [0.62285966]
 [0.71511626]
 [0.510514  ]
 [0.63853467]
 [0.5920097 ]
 [0.6836758 ]
 [0.83730686]
 [0.89214146]
 [0.73632735]
 [0.6996339 ]
 [0.8898366 ]
 [0.9459672 ]
 [0.42786235]
 [0.41356122]
 [0.690152  ]
 [0.7882622 ]
 [0.5336361 ]
 [0.59279907]
 [0.78511316]
 [0.59617263]
 [0.6505169 ]
 [0.5592191 ]
 [0.7501    ]
 [0.5602623 ]
 [0.72314435]
 [0.6473776 ]
 [0.58208126]
 [0.7021614 ]
 [0.7473672 ]
 [0.73853016]
 [0.9412117 ]
 [0.5183402 ]
 [0.64050627]
 [0.7015032 ]
 [0.67723143]
 [0.62363523]
 [0.6696618 ]
 [0.63286847]
 [0.5191433 ]
 [0.86951226]
 [0.68765587]
 [0.65486425]
 [0.5714667 ]
 [0.73552924]
 [0.73083216]
 [0.6817647 ]
 [0.6254139 ]
 [0.6869535 ]
 [0.8421723 ]
 [0.3485261 ]
 [0.58015203]
 [0.92588395]
 [0.7173929 ]
 [0.6431098 ]
 [0.80443305]
 [0.6188

In [109]:
np.sum(y_hat_test)

466.0